## vLLM setting

In [1]:
!nvidia-smi

import sys, torch
print("Python:", sys.version)
print("Torch:", torch.__version__)
print("Torch CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("Compute capability:", torch.cuda.get_device_capability(0))

Tue Jan  6 03:33:30 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["HF_HOME"] = "/content/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/content/hf_cache"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
print("HF_HOME =", os.environ["HF_HOME"])

HF_HOME = /content/hf_cache


In [3]:
!pip -q uninstall -y transformers tokenizers
!pip -q install -U "transformers==4.45.2" "tokenizers==0.20.3" "accelerate>=0.33.0"

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

In [5]:
%%capture
# 1) Cài PyTorch CUDA 12.1 (để tránh pip kéo nhầm torch CPU)
!pip -q install -U --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0

# 2) Cài vLLM (pin bản 0.6.6.post1) + các lib hay cần cho HF
!pip -q install -U \
  vllm==0.6.6.post1 \
  transformers accelerate sentencepiece safetensors tqdm


## Test vLLM

In [3]:
import transformers, vllm, torch

print("transformers:", transformers.__version__)
print("vllm:", vllm.__version__)
print("torch:", torch.__version__, "CUDA:", torch.version.cuda)

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


transformers: 4.45.2
vllm: 0.6.6.post1
torch: 2.5.1+cu124 CUDA: 12.4


In [ ]:
# from vllm import LLM, SamplingParams

# llm = LLM(
#     model="Qwen/Qwen3-0.6B",
#     tensor_parallel_size=1,
#     gpu_memory_utilization=0.9,
# )

# params = SamplingParams(temperature=0.0, max_tokens=32)
# out = llm.generate(["Trả lời ngắn gọn: 1+1=?"], params)
# print(out[0].outputs[0].text.strip())

In [2]:
!ls -lh /content/Private_Test_ref.json \
        /content/ground_truth_private_test.json \
        /content/qwen_infer.py \
        /content/evaluation.py

!mkdir -p /content/predictions /content/results /content/logs
!ls -lh /content/predictions /content/results /content/logs

-rw-r--r-- 1 root root  16K Jan  6 02:48 /content/evaluation.py
-rw-r--r-- 1 root root 3.8M Jan  6 02:46 /content/ground_truth_private_test.json
-rw-r--r-- 1 root root  14M Jan  6 02:47 /content/Private_Test_ref.json
-rw-r--r-- 1 root root 9.0K Jan  6 03:27 /content/qwen_infer.py
/content/logs:
total 12K
-rw-r--r-- 1 root root 714 Jan  6 03:23 qwen_infer_20260106_032358.log
-rw-r--r-- 1 root root 714 Jan  6 03:31 qwen_infer_20260106_033155.log
-rw-r--r-- 1 root root 714 Jan  6 03:37 qwen_infer_20260106_033747.log

/content/predictions:
total 0

/content/results:
total 0


## Qwen inference

#####Zero-shot

In [3]:
!python /content/qwen_infer.py \
  --input_file /content/Private_Test_ref.json \
  --output_file /content/predictions/predictions_qwen_zero.json \
  --model_name Qwen/Qwen2.5-7B-Instruct-AWQ \
  --mode zero-shot \
  --max_new_tokens 32 \
  --temperature 0.0 \
  --top_p 0.7 \
  --max_context_chars 3500 \
  --batch_size 4 \
  --tensor_parallel_size 1 \
  --gpu_memory_utilization 0.8 \
  --log_dir /content/logs

Streaming output truncated to the last 5000 lines.
Generating:  17%|███████▉                                      | 315/1826 [16:42<1:15:58,  3.02s/it]
Processed prompts:   0% 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
Processed prompts: 100% 4/4 [00:03<00:00,  1.29it/s, est. speed input: 660.93 toks/s, output: 41.17 toks/s]
Generating:  17%|███████▉                                      | 316/1826 [16:45<1:16:43,  3.05s/it]
Processed prompts:   0% 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
Processed prompts:  25% 1/4 [00:02<00:06,  2.10s/it, est. speed input: 239.04 toks/s, output: 3.82 toks/s]
Processed prompts: 100% 4/4 [00:03<00:00,  1.30it/s, est. speed input: 653.00 toks/s, output: 33.89 toks/s]
Generating:  17%|███████▉                                      | 317/1826 [16:49<1:16:52,  3.06s/it]
Processed prompts:   0% 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
Processed prompts:  25% 1/4 [00:

#####Few-shot

In [ ]:
!python /content/qwen_infer.py \
  --input_file /content/Private_Test_ref.json \
  --output_file /content/predictions/predictions_qwen_few.json \
  --model_name Qwen/Qwen2.5-7B-Instruct-AWQ \
  --mode few-shot \
  --max_new_tokens 32 \
  --temperature 0.0 \
  --top_p 0.7 \
  --max_context_chars 3500 \
  --batch_size 2 \
  --tensor_parallel_size 1 \
  --gpu_memory_utilization 0.8\
  --log_dir /content/logs

## Evaluation

#####Zero-shot

In [5]:
!python /content/evaluation.py \
  /content/ground_truth_private_test.json \
  /content/predictions/predictions_qwen_zero.json \
  --out-file /content/results/evaluation_qwen_zero.json \
  --log-dir /content/logs

2026-01-06 05:25:25 - INFO - Logger initialized. Log file: /content/logs/evaluation_20260106_052525.log
2026-01-06 05:25:25 - INFO - Loading dataset from /content/ground_truth_private_test.json
2026-01-06 05:25:25 - INFO - Loading predictions from /content/predictions/predictions_qwen_zero.json
2026-01-06 05:25:25 - INFO - No na_prob_file provided, using default 0.0 for all questions
2026-01-06 05:25:25 - INFO - Dataset contains 19 examples
2026-01-06 05:25:25 - INFO - Predictions contain 7301 entries
2026-01-06 05:25:26 - INFO - Evaluating on 3712 questions with predictions
2026-01-06 05:25:26 - INFO -   Has answer: 2596
2026-01-06 05:25:26 - INFO -   No answer: 1116
2026-01-06 05:25:26 - INFO - Evaluation Results:
2026-01-06 05:25:26 - INFO -   Exact Match: 1.48%
2026-01-06 05:25:26 - INFO -   F1 Score: 24.13%
2026-01-06 05:25:26 - INFO -   HasAns - Exact: 2.12%
2026-01-06 05:25:26 - INFO -   HasAns - F1: 34.50%
2026-01-06 05:25:26 - INFO -   NoAns - Exact: 0.00%
2026-01-06 05:25:26 